In [66]:
import numpy as np
import pygrib

import os
os.getcwd()

'/Users/jlfwong/code/hvac-sim-app/scripts/weather'

In [34]:
path = "./data/all-canada-2023.grib"

all_canada_2023_grbs = pygrib.open(path)

In [35]:
all_canada_2024_01_01_grbs = pygrib.open("./data/all-canada-2024-01-01.grib")

In [5]:
all_canada_2023_grbs.seek(0)
grb = all_canada_2023_grbs.read(1)[0]

grb.values.shape
lats, lons = grb.latlons()

(np.min(lats), np.max(lats), np.min(lons), np.max(lons))

(41.0, 58.0, -137.0, 308.0)

In [7]:
import json
ca_postal_codes = json.loads(open("./ca-postal-codes.json", "r").read())

def get_lat_lon(postal_code):
    info = ca_postal_codes[postal_code[:3]]
    return info["lat"], info["lng"]

get_lat_lon("K2A")

(45.3805, -75.7636)

In [8]:
import math


def relative_humidity(drybulb_temp_kelvin, dewpoint_temp_kelvin):
  # August-Roche-Magnus formula
  #
  # https://bmcnoldy.earth.miami.edu/Humidity.html
  # https://en.wikipedia.org/wiki/Clausius%E2%80%93Clapeyron_relation#Meteorology_and_climatology
  # 100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T)))
  # (Temperatures from this formula are in celcius)
  t = drybulb_temp_kelvin - 273.15
  td = dewpoint_temp_kelvin - 273.15
  return 100 * math.exp((17.625 * td) / (243.04 + td)) / math.exp((17.625 * t) / (243.04 + t))

print(relative_humidity(273.15 + (90 - 32) / 1.8, 273.15 + (63 - 32) / 1.8))


40.783503702475876


In [60]:
import requests

def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(url)
    for i in range(5):
        if response.status_code == 200:
            break

        # For some insane reason, sometimes the open elevation API gives 504s on certain
        # URLs, but appending 0s fixes the issue. Usually one is enough, but sometimes
        # two are needed.
        url += "0"
        response = requests.get(url)
    try:
        data = response.json()
    except Exception as e:
        print(f"Failed to retrieve elevation. {url}")
        raise e
    elevation = data["results"][0]["elevation"]
    return elevation

# elevation = get_elevation(lat, lon)  # m

In [61]:
def get_timezone(lat, lon):
    response = requests.get(f"http://timezonefinder.michelfe.it/api/0_{lon}_{lat}")
    try:
        location = response.json()
        timezone_str = location["tz_name"]
        return timezone_str
    except Exception as e:
        print(f"Failed to retrieve elevation. {url}")
        raise e

get_timezone(*get_lat_lon("K2A"))

'America/Toronto'

In [13]:
all_postal_codes = list(json.loads(open("ca-postal-codes.json").read()).keys())
len(all_postal_codes)

1651

In [50]:
def get_idx_for_lat_lon(lat, lon):
    # Find the grid point with closest lat/lon
    abslat = np.abs(lats-lat)
    abslon = np.abs(lons-lon)
    c = np.sqrt(np.add(np.square(abslat), np.square(abslon)))
    idx = np.argmin(c)
    return idx


In [45]:
from datetime import datetime
import pysolar
import json
import os

In [71]:
# Took 2m 9.7s -- 26s/code
# postal_codes = ["K2A", "V5K", "H3H", "R3T", "T6G"]

# Took 7m 58.8s for [:20] -- 24s/code
# Took 335m 21.5s for [:200] -- 100s/code (why?)
# Took 6m  5.4s for [200:20]
# Roughly 24s/postal

wind_u_key = '10 metre U wind component'
wind_v_key = '10 metre V wind component'
dewpoint_temp_key = '2 metre dewpoint temperature'
temp_key = '2 metre temperature'
cloud_cover_key = 'Total cloud cover'

# https://github.com/jeroenterheerdt/HAsmartirrigation/commit/c1cd7e96f81b41f0662c721af3d38b9a9
wind_speed_10m_to_2m_ratio = 4.87 / math.log((67.8 * 10) - 5.42)

era5_data = {}

def output_json_path(postal_code):
    return f'data/ca/2023-era5-{postal_code}.json'

def already_wrote_json(postal_code):
    return os.path.exists(output_json_path(postal_code))

filtered_postal_codes = [c for c in all_postal_codes if not already_wrote_json(c)]

print(f"{len(filtered_postal_codes)}/{len(all_postal_codes)} postal codes remaining")
print(f"Estimated time: {len(filtered_postal_codes) * (30 / 60 / 60)} hours")

codes_per_batch = 5
for postal_code_idx in range(0, len(filtered_postal_codes), codes_per_batch):
    postal_codes = filtered_postal_codes[postal_code_idx:postal_code_idx + codes_per_batch]
    print(datetime.now().strftime("%H:%M:%S"), "-", postal_codes)


    print(datetime.now().strftime("%H:%M:%S"), "- starting gathering location metadata")

    try:
        lat_lons = [get_lat_lon(code) for code in postal_codes]
        idx = [get_idx_for_lat_lon(lat, lon) for (lat, lon) in lat_lons]
        timezones = [get_timezone(lat, lon) for (lat, lon) in lat_lons]
        elevations = [get_elevation(lat, lon) for (lat, lon) in lat_lons]
    except Exception as e:
        print("Hit exception. Skipping")
        print(e)
        continue

    # We index all of the data for multiple postal code up-front because we want to
    # avoid doing multiple scans of the 2GB file
    print(datetime.now().strftime("%H:%M:%S"), "- started extracting data for grib")
    for grbs in [all_canada_2023_grbs, all_canada_2024_01_01_grbs]:
        grbs.seek(0)
        for grb in grbs.read():
            year = str(grb.dataDate)[:4]
            month = str(grb.dataDate)[4:6]
            day = str(grb.dataDate)[6:]

            time = '{:04d}'.format(grb.dataTime)
            hour = time[:2]
            minute = time[2:]

            dt = '{}-{}-{}T{}:{}:00+00:00'.format(year, month, day, hour, minute)
            for i, postal_code in enumerate(postal_codes):
                era5_data.setdefault(postal_code, {}).setdefault(dt, {})[grb.name] = grb.values.flat[idx[i]]

    results_by_postal_code = {}

    print(datetime.now().strftime("%H:%M:%S"), "- started constructing results")
    for i, postal_code in enumerate(postal_codes):
        results_by_postal_code[postal_code] = {
            "postalCode": postal_code,
            "timezoneName": timezones[i],
            "elevationMeters": elevations[i],
        }
        results = results_by_postal_code[postal_code]["weather"] = []
        lat, lon = lat_lons[i]

        for dt, vs in era5_data[postal_code].items():
            dt_obj = datetime.fromisoformat(dt)

            solar_altitude = pysolar.solar.get_altitude(lat, lon, dt_obj, elevations[i])
            solar_radiation = pysolar.radiation.get_radiation_direct(dt_obj, solar_altitude) if solar_altitude > 0 else 0

            temp = vs[temp_key]
            dewpoint_temp = vs[dewpoint_temp_key]
            rh = relative_humidity(temp, dewpoint_temp)

            wind_u = vs[wind_u_key]
            wind_v = vs[wind_v_key]
            wind_speed_10m = math.sqrt(wind_u * wind_u + wind_v * wind_v)
            wind_speed_2m = wind_speed_10m * wind_speed_10m_to_2m_ratio

            cloud_cover = vs[cloud_cover_key]

            row = {
                "datetime": dt_obj.isoformat(),
                "outsideAirTempF": float("{:.1f}".format((temp - 273.15) * 1.8 + 32)),
                "relativeHumidityPercent": float("{:.1f}".format(rh)),
                "windSpeedMph": float("{:.1f}".format(wind_speed_2m * 2.237)),
                "cloudCoverPercent": float("{:.1f}".format(cloud_cover * 100.0)),
                "solarIrradiance": {
                    "altitudeDegrees": float("{:.1f}".format(solar_altitude)),
                    "wattsPerSquareMeter": float("{:.1f}".format(solar_radiation))
                }
            }
            results.append(row)

    print(datetime.now().strftime("%H:%M:%S"), "- started writing output files")
    for postal_code, results in results_by_postal_code.items():
        with open(output_json_path(postal_code), 'w') as file:
            json.dump(results, file)

0/1651 postal codes remaining
Estimated time: 0.0 hours


In [103]:
# To compress files, run the following in the data/ca directory
#
# comm -23 <(ls *.json) <(ls *.json.br | sed 's/.br//g') | parallel --eta --bar brotli -o {}.br {}
#
# To upload files to s3:
#
# ls *.json | parallel --eta --bar aws s3 cp {}.br s3://hvac-sim-public/weather/ca/era/{} --content-encoding br